In [2]:
import streamlit as st
import dask.dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from streamlit_option_menu import option_menu

        
# Function to calculate metrics
@st.cache_data
def calculate_metrics(df, index_var):
    # Initialize the columns for burst and leak calculations
    df[f'bursts/km_{index_var}'] = 0
    df[f'leaks/km_{index_var}'] = 0

    # Calculate the total length and total km for each index_var value
    total_length = df.groupby(index_var)['LENGTH'].sum().compute()
    total_km = total_length / 1000

    # Calculate the count of bursts and leaks for each index_var value
    burst_counts = df[df['FAULT_TYPE'] == 'BURST'].groupby(index_var)['LENGTH'].count().compute()
    leak_counts = df[df['FAULT_TYPE'] == 'LEAK'].groupby(index_var)['LENGTH'].count().compute()

    # Calculate bursts/km and leaks/km
    for var in total_km.index:
        if var in burst_counts:
            df.loc[df[index_var] == var, f'bursts/km_{index_var}'] = burst_counts[var] / total_km[var]
        if var in leak_counts:
            df.loc[df[index_var] == var, f'leaks/km_{index_var}'] = leak_counts[var] / total_km[var]

    # Create a summary dataframe
    summary_df = df.groupby(index_var).agg({
        f'bursts/km_{index_var}': 'max',
        f'leaks/km_{index_var}': 'max',
        'LENGTH': 'sum'
    }).compute()

    # Rename the LENGTH column to total_km
    summary_df.rename(columns={'LENGTH': f'{index_var}_Km'}, inplace=True)
    summary_df[f'{index_var}_Km'] = summary_df[f'{index_var}_Km'] / 1000

    return summary_df

# Function to load and concatenate data from multiple files
@st.cache_data
def load_and_concatenate_data(uploaded_files):
    dataframes = []
    for uploaded_file in uploaded_files:
        try:
            # Load each file as a Dask dataframe
            df = dd.read_excel(uploaded_file, engine='openpyxl')
            dataframes.append(df)
        except Exception as e:
            st.error(f"Error loading file {uploaded_file.name}: {e}")
            return None
    # Concatenate all dataframes into a single Dask dataframe
    if dataframes:
        return dd.concat(dataframes, axis=0)
    else:
        return None

# Streamlit app
st.title('Water Bursts and Leaks Data Analysis')

# File uploader
uploaded_files = st.file_uploader("Choose Excel files", accept_multiple_files=True, type=["xlsx"])

if uploaded_files:
    # Load and concatenate data from selected files
    df = load_and_concatenate_data(uploaded_files)
    if df is not None:
        st.write(f"Uploaded files: {[file.name for file in uploaded_files]}")
        st.dataframe(df.head().compute())  # Display only the first few rows

        # User input for variable name
        selected_var = st.text_input('Enter Variable for Analysis')

        if selected_var:
            if selected_var in df.columns:
                # Calculate metrics for selected variable
                st.header(f'Analysis for {selected_var}')
                table_metrics = calculate_metrics(df, selected_var)
                st.dataframe(table_metrics)

                # Debug: Print column names
                st.write("Table Metrics Columns:", table_metrics.columns)

                # User selection for visualization
                plot_type = st.radio("Select plot type", ('Bursts', 'Leaks'))

                # Visualizations
                st.header('Visualizations')

                if plot_type == 'Bursts':
                    if f'bursts/km_{selected_var}' in table_metrics.columns:
                        st.write('Bursts per km')
                        fig1, ax1 = plt.subplots(figsize=(10, 6))
                        table_bursts_plot = table_metrics[[f'bursts/km_{selected_var}']]
                        sns.barplot(data=table_bursts_plot, ax=ax1, palette="viridis")
                        ax1.set_title('Bursts per km')
                        ax1.set_xlabel(selected_var)
                        ax1.set_ylabel('bursts/km')
                        ax1.legend(title='Bursts')
                        st.pyplot(fig1)
                    else:
                        st.error("No burst data available for the selected variable.")
                elif plot_type == 'Leaks':
                    if f'leaks/km_{selected_var}' in table_metrics.columns:
                        st.write('Leaks per km')
                        fig2, ax2 = plt.subplots(figsize=(10, 6))
                        table_leaks_plot = table_metrics[[f'leaks/km_{selected_var}']]
                        sns.barplot(data=table_leaks_plot, ax=ax2, palette="magma")
                        ax2.set_title('Leaks per km')
                        ax2.set_xlabel(selected_var)
                        ax2.set_ylabel('leaks/km')
                        ax2.legend(title='Leaks')
                        st.pyplot(fig2)
                    else:
                        st.error("No leak data available for the selected variable.")
            else:
                st.error(f"The variable '{selected_var}' is not found in the uploaded file.")
    else:
        st.error("Failed to load the file. Please check the file format and try again.")

2024-06-30 04:01:24.480 
  command:

    streamlit run C:\Users\21071252r\Anaconda3\envs\Latest\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
